In [1]:
import numpy as np
import xarray as xr
import pandas as pd
import copy
from datetime import datetime, timedelta
from keras.utils import to_categorical
# import visualkeras
# import tensorflow as tf
from sklearn.metrics import balanced_accuracy_score
import optuna
from optuna.samplers import TPESampler
import keras
from keras.callbacks import ModelCheckpoint
from sklearn.utils.class_weight import compute_class_weight
import sys
import os
import joblib
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import glob 

sys.path.append("/glade/u/home/jhayron/WR_Predictability/3_MLModels/")
from model_builders_v2 import *
from sklearn import datasets, ensemble
from sklearn.model_selection import RandomizedSearchCV

2023-10-17 14:07:55.389371: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
/glade/work/jhayron/conda-envs/cnn_wr/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Load outputs

In [2]:
week_out=0
week_out_str = f'week{week_out}'

wr_series = pd.read_csv('/glade/work/jhayron/Data4Predictability/WR_Series_20230824.csv',\
                index_col=0,names=['week0'],skiprows=1,parse_dates=True)
for wk in range(2,10):
    series_temp = copy.deepcopy(wr_series["week0"])
    series_temp.index = series_temp.index - timedelta(weeks = wk-1)
    series_temp.name = f'week{wk-1}'
    if wk==2:
        df_shifts = pd.concat([pd.DataFrame(wr_series["week0"]),pd.DataFrame(series_temp)],axis=1)  
    else:
        df_shifts = pd.concat([df_shifts,pd.DataFrame(series_temp)],axis=1)

# Load inputs

In [9]:
list_folders = np.sort(glob.glob('/glade/u/home/jhayron/WR_Predictability/4_PCA_Analysis/figures_2/*/'))

In [10]:
list_vars = [list_folders[i].split('/')[-2] for i in range(len(list_folders))]

In [11]:
list_vars

['IC_SODA',
 'IT_SODA',
 'MLD_SODA',
 'OHC100_SODA',
 'OHC200_SODA',
 'OHC300_SODA',
 'OHC50_SODA',
 'OHC700_SODA',
 'OLR_ERA5',
 'SD_ERA5',
 'SSH_SODA',
 'SST_OISSTv2',
 'SST_SODA',
 'STL_1m_ERA5',
 'STL_28cm_ERA5',
 'STL_7cm_ERA5',
 'STL_full_ERA5',
 'SWVL_1m_ERA5',
 'SWVL_28cm_ERA5',
 'SWVL_7cm_ERA5',
 'SWVL_full_ERA5',
 'U10_ERA5',
 'U200_ERA5',
 'Z500_ERA5']

In [12]:
indices_atmosphere = np.array([8,21,22,23])

In [13]:
dic_inputs = {}
for ivar in indices_atmosphere:
    dic_inputs[list_vars[ivar]] = pd.read_csv(f'{list_folders[ivar]}PC_{list_vars[ivar]}.csv',index_col=0,parse_dates=True)

In [14]:
# Create an empty DataFrame to store the combined data
combined_df = pd.DataFrame()

# Loop through the dictionary and concatenate the dataframes
for key, data in dic_inputs.items():
    # Convert the dictionary for the current key into a DataFrame
    df = pd.DataFrame(data)
    # Concatenate the current DataFrame with the combined DataFrame
    combined_df = pd.concat([combined_df, df], axis=1)

In [15]:
dic_inputs

{'OLR_ERA5':                    0         1         2         3         4         5  \
 1981-01-05 -1.260979 -0.117326 -1.237642  1.996850 -1.867071  1.527498   
 1981-01-08 -1.297153  0.106218 -2.044141  1.851592 -2.221492  1.766507   
 1981-01-12 -0.976288  0.376949 -1.989342  1.557745 -2.734780  1.866513   
 1981-01-15 -0.506496  0.347117 -1.996729  0.642193 -2.406635  1.755482   
 1981-01-19 -0.459188  0.158189 -2.371535  0.347430 -1.742322  1.278521   
 ...              ...       ...       ...       ...       ...       ...   
 2020-11-12  0.158695 -0.239138  0.590168  0.948768 -0.268944 -0.276295   
 2020-11-16  1.152207 -0.682926 -0.226729  1.314718 -1.074993 -0.659972   
 2020-11-19  1.496329 -1.092549  0.036611  1.696998 -1.809858 -0.838369   
 2020-11-23  0.271343 -1.498994  1.265712  1.828534 -1.732527 -0.665667   
 2020-11-26 -0.309994 -1.628059  1.228279  1.919033 -1.113822 -0.072148   
 
                    6         7         8         9        10        11  
 1981-01-05 

# Train model

In [16]:
fully_combined_df = pd.concat([combined_df,df_shifts[week_out_str]],axis=1)
fully_combined_df = fully_combined_df.dropna()

X_train = fully_combined_df['1980':'2015'].iloc[:,:-1].values
y_train = fully_combined_df['1980':'2015'].iloc[:,-1].values

# X_val = fully_combined_df['2011':'2015'].iloc[:,:-1].values
# y_val = fully_combined_df['2011':'2015'].iloc[:,-1].values

X_test = fully_combined_df['2016':'2020'].iloc[:,:-1].values
y_test = fully_combined_df['2016':'2020'].iloc[:,-1].values

In [17]:
# clf = ensemble.GradientBoostingClassifier()
# clf.fit(X_train, y_train)

# acc_train = clf.score(X_train, y_train)
# acc_val = clf.score(X_val, y_val)
# acc_test = clf.score(X_test, y_test)
# print("Accuracy train: {:.4f}".format(acc_train))
# print("Accuracy val: {:.4f}".format(acc_val))
# print("Accuracy test: {:.4f}".format(acc_test))

# optimize parameters for week 3

In [18]:
param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 4, 5, 6, 7],
    'min_samples_split': [2, 3, 4, 5],
    'min_samples_leaf': [1, 2, 3, 4],
    'subsample': [0.8, 0.9, 1.0],
    'max_features': ['sqrt', 'log2', None],
    'random_state': [42],  # Set to a specific value for reproducibility
    'criterion': ['friedman_mse', 'squared_error'],  # Splitting criterion
    'min_impurity_decrease': [0.0, 0.1, 0.2],  # Minimum impurity decrease for split
    'min_weight_fraction_leaf': [0.0, 0.1, 0.2],  # Minimum weighted fraction for leaf
}

In [21]:
# Create the GradientBoostingClassifier
clf = ensemble.GradientBoostingClassifier()

# Create the Grid Search object
grid_search = RandomizedSearchCV(clf, param_grid, cv=5, scoring='accuracy', 
                                 n_jobs=18,n_iter=100,return_train_score=True)

# Fit the model with training data
grid_search.fit(X_train, y_train)

# Get the best parameters and best estimator
best_params = grid_search.best_params_
best_clf = grid_search.best_estimator_
best_score = grid_search.best_score_

In [22]:
acc_train = best_clf.score(X_train, y_train)
# acc_val = best_clf.score(X_val, y_val)
acc_test = best_clf.score(X_test, y_test)
print("Accuracy train: {:.4f}".format(acc_train))
# print("Accuracy val: {:.4f}".format(acc_val))
print("Accuracy test: {:.4f}".format(acc_test))

Accuracy train: 0.7533
Accuracy test: 0.5410


# Iterate atmosphere, land, ocean

In [35]:
for ivar,var in enumerate(list_vars):
    print(ivar,var)

0 IC_SODA
1 IT_SODA
2 MLD_SODA
3 OHC100_SODA
4 OHC200_SODA
5 OHC300_SODA
6 OHC50_SODA
7 OHC700_SODA
8 OLR_ERA5
9 SD_ERA5
10 SSH_SODA
11 SST_OISSTv2
12 SST_SODA
13 STL_1m_ERA5
14 STL_28cm_ERA5
15 STL_7cm_ERA5
16 STL_full_ERA5
17 SWVL_1m_ERA5
18 SWVL_28cm_ERA5
19 SWVL_7cm_ERA5
20 SWVL_full_ERA5
21 U10_ERA5
22 U200_ERA5
23 Z500_ERA5


In [36]:
indices_atmosphere = np.array([8,21,22,23])
indices_land = np.array([9,13,16,17,20])
indices_ocean = np.array([0,1,2,4,6,10,12])

In [ ]:
for component in ['atm','ocn','lnd']:
    if component=='atm':
        dic_inputs = {}
        for ivar in indices_atmosphere:
            dic_inputs[list_vars[ivar]] = pd.read_csv(f'{list_folders[ivar]}PC_{list_vars[ivar]}.csv',index_col=0,parse_dates=True)
    elif component =='ocn':
        dic_inputs = {}
        for ivar in indices_ocean:
            dic_inputs[list_vars[ivar]] = pd.read_csv(f'{list_folders[ivar]}PC_{list_vars[ivar]}.csv',index_col=0,parse_dates=True)
    elif component =='lnd':
        dic_inputs = {}
        for ivar in indices_land:
            dic_inputs[list_vars[ivar]] = pd.read_csv(f'{list_folders[ivar]}PC_{list_vars[ivar]}.csv',index_col=0,parse_dates=True)
    print(dic_inputs.keys())
    # Create an empty DataFrame to store the combined data
    combined_df = pd.DataFrame()

    # Loop through the dictionary and concatenate the dataframes
    for key, data in dic_inputs.items():
        # Convert the dictionary for the current key into a DataFrame
        df = pd.DataFrame(data)
        # Concatenate the current DataFrame with the combined DataFrame
        combined_df = pd.concat([combined_df, df], axis=1)
    
    scores_test = []
    scores_val = []
    
    for week_out in np.arange(1,9):
        print(week_out)
        week_out_str = f'week{week_out}'
        fully_combined_df = pd.concat([combined_df,df_shifts[week_out_str]],axis=1)
        fully_combined_df = fully_combined_df.dropna()

        X_train = fully_combined_df['1980':'2015'].iloc[:,:-1].values
        y_train = fully_combined_df['1980':'2015'].iloc[:,-1].values

        X_test = fully_combined_df['2016':'2020'].iloc[:,:-1].values
        y_test = fully_combined_df['2016':'2020'].iloc[:,-1].values
        
        param_grid = {
            'n_estimators': [100, 200, 300],
            'learning_rate': [0.01, 0.1, 0.2],
            'max_depth': [3, 4, 5, 6, 7],
            'min_samples_split': [2, 3, 4, 5],
            'min_samples_leaf': [1, 2, 3, 4],
            'subsample': [0.5, 0.7, 0.9, 1],
            'max_features': ['sqrt', 'log2', None],
            'random_state': [42],  # Set to a specific value for reproducibility
            'criterion': ['friedman_mse', 'squared_error'],  # Splitting criterion
            'min_impurity_decrease': [0.0, 0.1, 0.2],  # Minimum impurity decrease for split
            'min_weight_fraction_leaf': [0.0, 0.1, 0.2],  # Minimum weighted fraction for leaf
        }
        # Create the GradientBoostingClassifier
        clf = ensemble.GradientBoostingClassifier()

        # Create the Grid Search object
        grid_search = RandomizedSearchCV(clf, param_grid, cv=5, scoring='accuracy', n_jobs=18,n_iter=100,return_train_score=True)

        # Fit the model with training data
        grid_search.fit(X_train, y_train)
        np.save(f'results/{component}_cv_results',grid_search.cv_results_)
        # Get the best parameters and best estimator
        best_params = grid_search.best_params_
        best_clf = grid_search.best_estimator_
        best_score = grid_search.best_score_
        
        scores_test.append(best_clf.score(X_test, y_test))
        scores_val.append(best_score)
        results = pd.DataFrame(np.array([best_clf.predict(X_test),y_test]).T,
                     index=fully_combined_df['2016':'2020'].index,
                     columns=['y_predicted','y_test'])
        
        np.save(f'results/{component}_scores_test',np.array(scores_test))
        np.save(f'results/{component}_scores_val',np.array(scores_val))
        print(best_score)
        print(scores_test[-1])
        results.to_csv(f'results/results_{component}_{week_out_str}.csv')

dict_keys(['OLR_ERA5', 'U10_ERA5', 'U200_ERA5', 'Z500_ERA5'])
1
0.3417255401682814
0.3411764705882353
2
0.25657140715477017
0.2421259842519685
3
0.23932087776174504
0.17786561264822134
4
0.2428881434701947
0.21825396825396826
5
0.24314749920356798
0.2549800796812749
6
0.24972883833367687
0.226
7
0.240142420778442
0.1746987951807229
8
0.24014204598691977
0.17540322580645162
dict_keys(['IC_SODA', 'IT_SODA', 'MLD_SODA', 'OHC200_SODA', 'OHC50_SODA', 'SSH_SODA', 'SST_SODA'])
1
0.25163990030545513
0.31176470588235294
2
0.239320877761745
0.17716535433070865
3
0.23932087776174504
0.17786561264822134
4
0.23959485036448475
0.1765873015873016
5
0.23959485036448475
0.17729083665338646
6
0.23986844817570224
0.176
7
0.240142420778442
0.1746987951807229
8
0.24014204598691977
0.17540322580645162
dict_keys(['SD_ERA5', 'STL_1m_ERA5', 'STL_full_ERA5', 'SWVL_1m_ERA5', 'SWVL_full_ERA5'])
1
0.26259130858459984
0.22941176470588234
2
0.2532796132151491
0.23818897637795275
3
